## LinkML and SQL Alchemy Tutorial

This tutorial walks through the use of LinkML in combination with SQLite3 and SQL Alchemy.

The steps are:

 - Create a SQL schema from a LinkML schema (the standard PersonInfo example)
 - Generate SQL Alchemy "Models" from the same LinkML schema
 - Create instances of these Models and insert into database
 - Query using direct SQL
 - Query using SQL Alchemy ORM

## PersonInfo schema

We use the standard PersonInfo schema bundled in the LinkML repo:

In [29]:
!cat ../examples/PersonSchema/personinfo.yaml

id: https://w3id.org/linkml/examples/personinfo
name: personinfo
description: |-
  Information about people, based on [schema.org](http://schema.org)
license: https://creativecommons.org/publicdomain/zero/1.0/
default_curi_maps:
  - semweb_context
imports:
  - linkml:types
prefixes:
  personinfo: https://w3id.org/linkml/examples/personinfo/
  linkml: https://w3id.org/linkml/
  schema: http://schema.org/
  rdfs: http://www.w3.org/2000/01/rdf-schema#
  prov: http://www.w3.org/ns/prov#
  GSSO: http://purl.obolibrary.org/obo/GSSO_
  famrel: https://example.org/FamilialRelations#
  # DATA PREFIXES
  P: http://example.org/P/
  ROR: http://example.org/ror/
  CODE: http://example.org/code/
  GEO: http://example.org/geoloc/
default_prefix: personinfo
default_range: string

emit_prefixes:
  - rdf
  - rdfs
  - xsd
  - skos

classes:

  NamedThing:
    description: >-
      A generic grouping for any identifiable entity
    slots:
      - id
      - name
      - description
      - image
    close

## Create a working directory

For demonstration purposes we will put all generated artefacts in a folder `tmp`.

Note that if you were doing this in a real project, you should use the linkml project cookiecutter
to create a standard project structure

In [30]:
!mkdir -p tmp

## Generate SQL CREATE TABLE statements

We use the [gen-sqlddl](https://linkml.io/linkml/generators/sqlddl.html) command to create a SQL DDL (CREATE TABLE statements):

In [127]:
!gen-sqlddl ../examples/PersonSchema/personinfo.yaml > tmp/personinfo.ddl.sql

INFO:root:Loading schema https://w3id.org/linkml/types from ../examples/PersonSchema/personinfo.yaml
INFO:root:Added primary key HasAliases.id
INFO:root:Added primary key Address.id
INFO:root:Added primary key Event.id
INFO:root:Added primary key Relationship.id
INFO:root:Added primary key FamilialRelationship.id
INFO:root:Added primary key EmploymentEvent.id
INFO:root:Added primary key MedicalEvent.id
INFO:root:Added primary key WithLocation.id
INFO:root:Added primary key Container.id


In [133]:
!grep -A20 'CREATE TABLE "Person"' tmp/personinfo.ddl.sql

CREATE TABLE "Person" (
	primary_email TEXT, 
	birth_date TEXT, 
	age_in_years INTEGER, 
	gender VARCHAR(17), 
	id TEXT, 
	name TEXT, 
	description TEXT, 
	image TEXT, 
	"Container_id" TEXT, 
	current_address_id TEXT, 
	PRIMARY KEY (id), 
	FOREIGN KEY("Container_id") REFERENCES "Container" (id), 
	FOREIGN KEY(current_address_id) REFERENCES "Address" (id)
);
CREATE TABLE "Organization" (
	mission_statement TEXT, 
	founding_date TEXT, 
	founding_location TEXT, 
	id TEXT, 
	name TEXT, 


### Create an empty database

Ensuring we start with a fresh database, we initialize it with CREATE TABLE STATEMENTS

In [52]:
!test -f tmp/tmp.db && rm tmp/tmp.db

In [53]:
!cat tmp/personinfo.ddl.sql | sqlite3 tmp/tmp.db 

In [140]:
!sqlite3 tmp/tmp.db ".schema Person" ".exit"

CREATE TABLE IF NOT EXISTS "Person" (
	primary_email TEXT, 
	birth_date TEXT, 
	age_in_years INTEGER, 
	gender VARCHAR(17), 
	id TEXT, 
	name TEXT, 
	description TEXT, 
	image TEXT, 
	"Container_id" TEXT, 
	current_address_id TEXT, 
	PRIMARY KEY (id), 
	FOREIGN KEY("Container_id") REFERENCES "Container" (id), 
	FOREIGN KEY(current_address_id) REFERENCES "Address" (id)
);


## Generate SQLA Models

We use the [gen-sqla](https://linkml.io/linkml/generators/sqlalchemy.html) command to create a SQLA Object Model

In [31]:
!gen-sqla ../examples/PersonSchema/personinfo.yaml > tmp/personinfo_sqla.py

INFO:root:Loading schema https://w3id.org/linkml/types from ../examples/PersonSchema/personinfo.yaml
INFO:root:Added primary key HasAliases.id
INFO:root:Added primary key Address.id
INFO:root:Added primary key Event.id
INFO:root:Added primary key Relationship.id
INFO:root:Added primary key FamilialRelationship.id
INFO:root:Added primary key EmploymentEvent.id
INFO:root:Added primary key MedicalEvent.id
INFO:root:Added primary key WithLocation.id
INFO:root:Added primary key Container.id
INFO:root:Package for dataclasses ==  personinfo


In [139]:
!cat tmp/personinfo_sqla.py


from sqlalchemy import Column, Index, Table, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.sql.sqltypes import *
from sqlalchemy.orm import declarative_base
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()
metadata = Base.metadata


class NamedThing(Base):
    """
    A generic grouping for any identifiable entity
    """
    __tablename__ = 'NamedThing'
    
    id = Column(Text(), primary_key=True)
    name = Column(Text())
    description = Column(Text())
    image = Column(Text())
    
    
    def __repr__(self):
        return f"NamedThing(id={self.id},name={self.name},description={self.description},image={self.image},)"
        
    
        
    


class HasAliases(Base):
    """
    A mixin applied to any class that can have aliases/alternateNames
    """
    __tablename__ = 'HasAliases'
    
    id = Column(Integer(), primary_key=True, autoincrement=True )
    
    
    aliases_rel = relationship( "HasAliasesAliase

## Populate the SQLA Model Objects

We will import our object model (remember in production code we would not put this in `tmp`).

We will also use the Jupyter builtin help to introspect a class - although the builtin SQLA methods make this
a bit verbose and messy

In [56]:
from tmp.personinfo_sqla import Person
help(Person)

Help on class Person in module tmp.personinfo_sqla:

class Person(NamedThing)
 |  Person(**kwargs)
 |  
 |  A person (alive, dead, undead, or fictional).
 |  
 |  Method resolution order:
 |      Person
 |      NamedThing
 |      sqlalchemy.orm.decl_api.Base
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      A simple constructor that allows initialization from kwargs.
 |      
 |      Sets attributes on the constructed instance using the names and
 |      values in ``kwargs``.
 |      
 |      Only keys that are present as
 |      attributes of the instance's class are allowed. These could be,
 |      for example, any mapped columns or relationships.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  Container_id
 |  
 |  age_in_years
 |  
 |  aliases_rel
 |  
 |  birth_date
 |  
 |  current_address
 |  
 |  current_ad

### Create a SQLA session

This is a bit involved, but this is all standard for SQLA

In [58]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [59]:
engine = create_engine("sqlite:///tmp/tmp.db")

In [60]:
session_class = sessionmaker(bind=engine)
session = session_class()

### Create an object and add it

We will populate the database with a single person, then do a sanity check SQL query

In [94]:
p = Person(id="P1", name="Rosario", age_in_years=33)

In [61]:
session.add(p)

In [62]:
session.commit()

In [63]:
!sqlite3 tmp/tmp.db "SELECT * FROM Person;" ".exit"

||33||P1|Rosario||||


## Foreign Keys

Now we demonstrate use of foreign keys behind the scenes.

In the LinkML model, FamilialRelationship is *inlined*, so each Person can have zero to many embedded relationships
to other people.

We will append a relationship to another person (P2 - note this person is not yet in the database,
but it's OK as SQLite3 doesn't enforce referential integrity)

In [141]:
from tmp.personinfo_sqla import FamilialRelationship

In [67]:
p.has_familial_relationships.append(FamilialRelationship(related_to="P2", type="SIBLING_OF"))

In [68]:
session.commit()

In [69]:
!sqlite3 tmp/tmp.db "SELECT * FROM FamilialRelationship;" ".exit"

1|||P2|SIBLING_OF|P1


### Generating synthetic data

Next we will write some simple code for making synthetic people

In [143]:
%pip install names

Note: you may need to restart the kernel to use updated packages.


In [114]:
import names
import random

last_id = 2
def random_person() -> Person:
    """
    Create a person with a random name and random set of relationships.
    
    Note that this is just for demo purposes. If we wanted to be more advanced we
    could make use of SIBLING_OF to make a more realistic distribution of last name
    matches, and we could ensure that SIBLING_OF is symmetric...
    """
    global last_id
    last_id += 1
    pid = f"P{last_id}"
    person = Person(id=pid, name=names.get_full_name(), age_in_years=random.randint(30,40))
    for n in range(0, random.randint(0, 3)):
        rel = FamilialRelationship(related_to=f"P{last_id+n}", type="SIBLING_OF")
        person.has_familial_relationships.append(rel)
    return person

In [144]:
persons = [random_person() for x in range(1,100)]

### Container Object

The PersonInfo schema has a class Container whose sole function is to contain all top level objects.

We will populate our container with our 100 artificial people.

In [145]:
from tmp.personinfo_sqla import Container

In [146]:
container = Container(persons=persons)

In [147]:
session.add(container)

In [148]:
session.commit()

In [149]:
!sqlite3 tmp/tmp.db "SELECT * FROM FamilialRelationship LIMIT 10;" ".exit"

1|||P2|SIBLING_OF|P1
2|||P5|SIBLING_OF|P5
3|||P6|SIBLING_OF|P6
4|||P7|SIBLING_OF|P7
5|||P8|SIBLING_OF|P7
6|||P9|SIBLING_OF|P7
7|||P8|SIBLING_OF|P8
8|||P9|SIBLING_OF|P8
9|||P10|SIBLING_OF|P8
10|||P9|SIBLING_OF|P9


## Use of SQLA ORM to query

Next we will demonstrate use of the SQLA Object-Relational Mapper to query for the first 10 objects in the database.

Note that the ORM takes care of writing queries involving joins for us, so inlined objects are retrieved naturally

In [126]:
for p in session.query(Person)[0:10]:
    print(p.name)
    for frel in p.has_familial_relationships:
        print(f"  {frel.type} {frel.related_to}")

Rosario
  SIBLING_OF P2
Jerome Horn
Carl Dawson
  SIBLING_OF P5
Ann Flores
  SIBLING_OF P6
Jolene Chaplin
  SIBLING_OF P7
  SIBLING_OF P8
  SIBLING_OF P9
Thelma Lindo
  SIBLING_OF P8
  SIBLING_OF P9
  SIBLING_OF P10
Lily Jordan
  SIBLING_OF P9
  SIBLING_OF P10
  SIBLING_OF P11
Mildred Sharp
  SIBLING_OF P10
  SIBLING_OF P11
Rana Dyson
  SIBLING_OF P11
  SIBLING_OF P12
  SIBLING_OF P13
William Hanover
  SIBLING_OF P12


## Bridging SQLA Models with LinkML Dataclasses or Pydantic

If you are already familiar with LinkML use with Python, you will likely be familiar with:

- [gen-python](https://linkml.io/linkml/generators/python.html)
- [gen-pydantic](https://linkml.io/linkml/generators/pydantic.html)

Which create dataclasses and pydantic object models respectively.

Unfortunately there is no one single standard for object models in Python. Dataclasses and Pydantic have a lot of advantages over SQLA models, but they don't work as well in conjunction with the ORM.

To accommodate this, the LinkML python toolchain has bridging code that will map objects from one system to another.

In [150]:
!gen-pydantic ../examples/PersonSchema/personinfo.yaml > tmp/personinfo.py

INFO:root:Loading schema https://w3id.org/linkml/types from ../examples/PersonSchema/personinfo.yaml


In [151]:
import tmp.personinfo as personinfo

In [167]:
personinfo.FamilialRelationship(type=personinfo.FamilialRelationshipType("SIBLING_OF")).dict()

{'started_at_time': None,
 'ended_at_time': None,
 'related_to': None,
 'type': <FamilialRelationshipType.SIBLING_OF: 'SIBLING_OF'>}

In [152]:
def random_person_pydantic() -> personinfo.Person:
    """
    Create a person with a random name and random set of relationships.
    
    Note that this is just for demo purposes. If we wanted to be more advanced we
    could make use of SIBLING_OF to make a more realistic distribution of last name
    matches, and we could ensure that SIBLING_OF is symmetric...
    """
    global last_id
    last_id += 1
    pid = f"P{last_id}"
    person = personinfo.Person(id=pid, name=names.get_full_name(), age_in_years=random.randint(30,40))
    for n in range(0, random.randint(0, 3)):
        rel = personinfo.FamilialRelationship(related_to=f"P{last_id+n}", type="SIBLING_OF")
        person.has_familial_relationships.append(rel)
    return person

In [153]:
random_person_pydantic()

Person(id='P203', name='Jose Stover', description=None, image=None, aliases=[], primary_email=None, birth_date=None, age_in_years=39, gender=None, current_address=None, has_employment_history=[], has_familial_relationships=[FamilialRelationship(started_at_time=None, ended_at_time=None, related_to='P203', type=<FamilialRelationshipType.SIBLING_OF: 'SIBLING_OF'>), FamilialRelationship(started_at_time=None, ended_at_time=None, related_to='P204', type=<FamilialRelationshipType.SIBLING_OF: 'SIBLING_OF'>), FamilialRelationship(started_at_time=None, ended_at_time=None, related_to='P205', type=<FamilialRelationshipType.SIBLING_OF: 'SIBLING_OF'>)], has_medical_history=[])

In [159]:
container = personinfo.Container(persons=[random_person_pydantic() for x in range(1,100)])

In [155]:
from linkml.utils.sqlutils import SQLStore

In [156]:
store = SQLStore()

In [157]:
store.module = personinfo

In [160]:
container_sqla = store.to_sqla(container)

In [162]:
type(container_sqla)

tmp.personinfo.Container